In [ ]:
!pip install segmentation-models
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 KB 3.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade --no-cache-dir gdown
!gdown --id 1-Vm4Y4YTPSH_zM3xSTzB9RQvJqiVFwaw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-Vm4Y4YTPSH_zM3xSTzB9RQvJqiVFwaw
To: /content/Tumor.zip
100% 156M/156M [00:02<00:00, 52.1MB/s]


In [ ]:
!pip install patool
import patoolib
patoolib.extract_archive('Tumor.zip')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 KB 7.3 MB/s eta 0:00:00
patool: Extracting Tumor.zip ...
patool: running /usr/bin/7z x -o./Unpack_7n8wsy0b -- Tumor.zip
patool: ... Tumor.zip extracted to `Tumor'.


'Tumor'

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import glob
import cv2
import os
from sklearn import preprocessing
from sklearn.metrics import accuracy_score


In [ ]:
SIZE = 128

train_images = []
train_labels = [] 

for directory_path in glob.glob("/content/Tumor/Training/*"):
    label = directory_path.split("/")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)

train_images = np.array(train_images)
train_labels = np.array(train_labels)

test_images = []
test_labels = [] 
for directory_path in glob.glob("/content/Tumor/Testing/*"):
    fruit_label = directory_path.split("/")[-1]
    print(fruit_label)
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)


pituitary
meningioma
glioma
notumor
pituitary
meningioma
glioma
notumor


In [ ]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(SIZE, SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(4)
])


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
history = model.fit(train_images, train_labels_encoded, epochs=100,
                    validation_data=(test_images, test_labels_encoded))


Epoch 1/100
179/179 [==============================] - 3s 17ms/step - loss: 0.2168 - accuracy: 0.9219 - val_loss: 0.3648 - val_accuracy: 0.8688
Epoch 2/100
179/179 [==============================] - 3s 17ms/step - loss: 0.1767 - accuracy: 0.9366 - val_loss: 0.2739 - val_accuracy: 0.8970
Epoch 3/100
179/179 [==============================] - 3s 17ms/step - loss: 0.1409 - accuracy: 0.9499 - val_loss: 0.3215 - val_accuracy: 0.8825
Epoch 4/100
179/179 [==============================] - 3s 17ms/step - loss: 0.1147 - accuracy: 0.9599 - val_loss: 0.2492 - val_accuracy: 0.9100
Epoch 5/100
179/179 [==============================] - 3s 16ms/step - loss: 0.0810 - accuracy: 0.9708 - val_loss: 0.2037 - val_accuracy: 0.9359
Epoch 6/100
179/179 [==============================] - 3s 16ms/step - loss: 0.0475 - accuracy: 0.9839 - val_loss: 0.2458 - val_accuracy: 0.9275
Epoch 7/100
179/179 [==============================] - 3s 17ms/step - loss: 0.0948 - accuracy: 0.9692 - val_loss: 0.2250 - val_accuracy:

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels_encoded, verbose=2)
print('\nTest accuracy:', test_acc)

41/41 - 0s - loss: 0.4498 - accuracy: 0.9641 - 260ms/epoch - 6ms/step

Test accuracy: 0.9641494750976562


In [ ]:
# Install LightGBM with GPU support and import necessary libraries
!pip install numpy==1.19.3
!git clone --recursive https://github.com/Microsoft/LightGBM
%cd /content/LightGBM
!mkdir build
%cd build
!cmake -DUSE_GPU=1 /content/LightGBM
!make -j$(nproc)
!sudo apt-get -y install python3-pip
!sudo -H python3 -m pip install setuptools pandas numpy scipy scikit-learn -U
!sudo -H python3 -m pip install wheel
!sudo -H python3 -m pip install graphviz lightgbm



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 89.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.24.2:
      Successfully uninstalled numpy-1.24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2022.12.0 requires numpy>=1.20, but you have numpy 1.19.3 which is incompatible.
xarray-einstats 0.5.1 requires numpy>=1.20, but you have numpy 1.19.3 which is incompatible.
tensorflow 2.11.0 requires numpy>=1.20, but you have numpy 1.19.3 which is incompatible.
scipy 1.10.0 requires numpy<1.27.0,>=1.19.5, but you have numpy 1.19.3 which is incompatible.
pandas 1.5.3 requires numpy>=1.20.3; python_version < "3.10", but you have numpy 1.19.3 which is incompatible.
jaxlib 0.3.25+cuda1

In [ ]:
# Install LightGBM with GPU support and import necessary libraries
!pip install numpy==1.19.3
!git clone --recursive https://github.com/Microsoft/LightGBM
%cd /content/LightGBM
!mkdir build
%cd build
!cmake -DUSE_GPU=1 /content/LightGBM
!make -j$(nproc)
!sudo apt-get -y install python3-pip
!sudo -H python3 -m pip install setuptools pandas numpy scipy scikit-learn -U
!sudo -H python3 -m pip install wheel
!sudo -H python3 -m pip install graphviz lightgbm


import numpy as np
import lightgbm as lgb
import glob
import cv2
import os
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

# Set image size
SIZE = 128

# Load and preprocess training data
train_images = []
train_labels = [] 

for directory_path in glob.glob("/content/Tumor/Training/*"):
    label = directory_path.split("/")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img.flatten())
        train_labels.append(label)

train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Load and preprocess testing data
test_images = []
test_labels = [] 
for directory_path in glob.glob("/content/Tumor/Testing/*"):
    fruit_label = directory_path.split("/")[-1]
    print(fruit_label)
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img.flatten())
        test_labels.append(fruit_label)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Encode labels using LabelEncoder
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

# Create LightGBM datasets
lgb_train = lgb.Dataset(train_images, train_labels_encoded)
lgb_test = lgb.Dataset(test_images, test_labels_encoded)

# Set parameters for LightGBM
params = {
    'objective': 'multiclass',
    'num_class': 4,
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'device': 'gpu'
}

# Train LightGBM model
num_boost_round = 100
bst = lgb.train(params, lgb_train, num_boost_round=num_boost_round)

# Make predictions on test set
y_pred = bst.predict(test_images)
y_pred = np.argmax(y_pred, axis=1)

# Compute accuracy score
accuracy = accuracy_score(test_labels_encoded, y_pred)
print('Test accuracy:', accuracy)


AttributeError: ignored